In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('train.csv')
df.head()

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,...,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,...,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,...,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,...,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,...,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0,1.0


In [3]:
column_categorical=list(df.select_dtypes(include='object').columns)
column_numerical=list(df.select_dtypes(exclude='object').columns)
column_numerical.remove('id')
column_numerical.remove('diagnosed_diabetes')
len(column_categorical),len(column_numerical)

(6, 18)

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(df[column_numerical])
df[column_numerical]=scaler.transform(df[column_numerical])

In [5]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
encoder.fit(df[column_categorical])

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [6]:
column_enc=list(encoder.get_feature_names_out())
len(column_enc)

24

In [8]:
df[column_enc]=encoder.transform(df[column_categorical])

In [9]:
X_train=df[column_enc+column_numerical]
y_train=df['diagnosed_diabetes']

In [16]:
df_test=pd.read_csv('test.csv')

#Scaling
df_test[column_numerical]=scaler.transform(df_test[column_numerical])

#Encoding
df_test[column_enc]=encoder.transform(df_test[column_categorical])

X_test=df_test[column_enc+column_numerical]

In [17]:
import xgboost as xgb


In [35]:
model_xgb = xgb.XGBRegressor(
    n_estimators=2000,
    learning_rate=0.02,
    max_depth=15,
    subsample=0.9,
    colsample_bytree=0.7,
    objective="reg:squarederror"
)

In [36]:
model_xgb.fit(X_train,y_train)
model_xgb.score(X_train,y_train)

0.9717345730930418

In [33]:
yp_xgb=model_xgb.predict(X_test)

In [34]:
df_xgb=pd.DataFrame({'id':df_test['id'],'diagnosed_diabetes':yp_xgb})
df_xgb.to_csv('submit_xgb.csv',index=False)

In [26]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [28]:
model_adaboostDT=AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=9,min_samples_leaf=8),n_estimators=100)
model_adaboostDT.fit(X_train,y_train)
yp_adaboostDT=model_adaboostDT.predict(X_test)
model_adaboostDT.score(X_train,y_train)

0.6959757142857143

In [30]:
df_ada=pd.DataFrame({'id':df_test['id'],'diagnosed_diabetes':yp_adaboostDT})
df_ada.to_csv('submit_ada.csv',index=False)